In [1]:
import random
import numpy as np
import itertools
import copy

## Convert x to S

In [2]:
def convert_x_to_S(x):
    x_bar = 0.5*np.ones(len(x))
    s = x >= x_bar
    S = []

    for i in range(len(s)):

        if s[i] == True:
            S.append(i+1)
    
    return S

### F(x) without sampling (not used)

In [3]:
def F_without_sampling(x, f, n, t):
    
    A = [np.array(i) for i in itertools.product([0, 1], repeat = n)]
    
    val_S = 0

    for i in range(len(A)):

        prod_in_s = 1
        prod_not_in_s = 1
        S = []

        for j in range(len(A[i])):
            if A[i][j] == 1:
                prod_in_s = prod_in_s*x[j]
                S.append(j+1)
            else:
                prod_not_in_s = prod_not_in_s*(1-x[j])

        val_S = val_S + f(S)*prod_in_s*prod_not_in_s
    
    return val_S

### Approximation of F(x) using sampling 
define t

In [4]:
def F(x, f, n, t):
    
    sum_R = 0
    
    for i in range(t):
            
        x_bar = np.random.uniform(0,1, n)
        r_t = x >= x_bar
        R_t = []
            
        for i in range(len(r_t)):
            
            if r_t[i] == True:
                R_t.append(i+1)
                
        sum_R = sum_R + f(R_t)

    return sum_R/t

In [5]:
def get_gradient_F_for_i(F, x, f, n, i, t):


    x_without_i = copy.deepcopy(x)
    x_without_i[i] = 0.0

    x_with_i = copy.deepcopy(x)
    x_with_i[i] = 1.0

    # print('x with xi: ', x_with_i)
    # print('x without xi: ', x_without_i)

    df_dxi = F(x_with_i, f, n, t) - F(x_without_i, f, n, t)
    return df_dxi

In [6]:
def get_gradient_F(F, x, f, t):
    
    n = len(x)
    
    grad = np.zeros(len(x))
    
    for i in range(len(x)):
        
        grad[i] = get_gradient_F_for_i(F, x, f, n, i, t)
    
    return grad

## Testing

Define the size of the problem (n) 

In [7]:
n = 10
N = [(i+1) for i in range(n)]

Define an x vector

In [8]:
x_init = np.random.uniform(0,1, n)
print("x: ", x)

NameError: name 'x' is not defined

The corresponding S vector

In [ ]:
S_init = convert_x_to_S(x_init)
S_init

In [ ]:
t = int(2**n/4)

## Constant function f(s)

In [ ]:
def f_constant(S):
    return 5

### Values of the function

Actual value

In [ ]:
f_constant(S)

Multi-linear extenstion

In [ ]:
F_without_sampling(x, f_constant, n, t)

Mutli-linear extension with sampling

In [ ]:
F(x, f_constant, n, t)

### Values of the gradient using multi-linear extension

without sampling

In [ ]:
get_gradient_F(F_without_sampling, x, f_constant, t)

With sampling

In [ ]:
get_gradient_F(F, x, f_constant, t)

## Linear function f(S)

In [ ]:
a = np.random.uniform(-1,1, n)
a

In [ ]:
def f_linear(S):
    
    #convert S to 0,1
    s_hat = np.zeros(n)
    
    for i in range(len(S)):
        
        s_hat[S[i]-1] = 1
        
    return np.dot(a,s_hat)

### Values of the function

Actual value

In [ ]:
f_linear(S)

Multi-linear extenstion

In [ ]:
F_without_sampling(x, f_linear, n, t)

Mutli-linear extension with sampling

In [ ]:
F(x, f_linear, n, t)

### Values of the gradient using multi-linear extension

without sampling

In [ ]:
get_gradient_F(F_without_sampling, x, f_linear, t)

With sampling

In [ ]:
get_gradient_F(F, x, f_linear, t)

## Polynomial f(S) function

In [ ]:
def f_polynomial(S):
    
    #convert S to 0,1
    s_hat = np.zeros(n)
    
    for i in range(len(S)):
        
        s_hat[S[i]-1] = 1
        
    a_hat = s_hat*a
    
    return np.dot(a_hat,a_hat)

### Values of function

Actual value

In [ ]:
f_polynomial(S)

Multi-linear extension 

In [ ]:
F_without_sampling(x, f_polynomial, n, t)

Multi-linear extenstion with sampling

In [ ]:
F(x, f_polynomial, n, t)

### Values of gradients using multi-linear approximation

Without sampling

In [ ]:
get_gradient_F(F_without_sampling, x, f_polynomial, t)

With sampling

In [ ]:
get_gradient_F(F, x, f_polynomial, t)

# Testing Gradient Ascent

## Find maximum using gradient ascent

In [ ]:
def gradient_ascent(F, x, f, n, alpha, t, epsilon):
    
    x_init = copy.deepcopy(x)
    sum_init = F(x, f, n, t)
    # key values to be used
    sum_update = 0
    iter = 0
    sum_temp = copy.deepcopy(sum_init)

    # start updating the parameters x with iterative gradients
    while np.abs(sum_temp - sum_update) > epsilon:
        iter += 1
        sum_temp = F(x, f, n, t)

        for i in range(n):
            grad_i = get_gradient_F_for_i(F, x, f, n, i, t)
            x[i] = np.minimum(x[i] + alpha * grad_i, 1.0)
            x[i] = np.maximum(x[i], 0.0)

        sum_update = F(x, f, n, t)

    print('Iterations: ', iter, '\n')
    print('Initial F: ', sum_init)
    print('Initial x: ', x_init, '\n')
    print('Final F: ', sum_update)
    print('Final x: ', x, '\n')
    return iter, sum_update, x

## Go through all the possible S and find the maximum

In [ ]:
def actual_max(f,n):
    
    A = [np.array(i) for i in itertools.product([0, 1], repeat = n)]
    
    max_val_S = 0
    argmax_S = []

    for i in range(len(A)):

        S = []

        for j in range(len(A[i])):
            if A[i][j] == 1:
                S.append(j+1)

        val_S = f(S)
        
        if val_S > max_val_S:
            max_val_S = val_S
            argmax_S = S
            print(i,S,val_S)
    
    return max_val_S, argmax_S
    

In [ ]:
# stepsize for gradient ascent
alpha = 0.0001
epsilon = 10**(-5)

### For f_linear

In [ ]:
x_initial = np.random.uniform(0,1, n)
S_init = convert_x_to_S(x_initial)

In [ ]:
print(x_initial,S_init)

In [ ]:
_,_,x_final = gradient_ascent(F, x_initial, f_linear, n, alpha, t, epsilon)
S_final_grad_ascent = convert_x_to_S(x_final)

In [ ]:
print(S_final_grad_ascent)

In [ ]:
max_val,S_final_actual_max = actual_max(f_linear, n)

In [ ]:
max_val,S_final_actual_max

We get the same output from the gradient descent as the exact output

### For f_polynomial

In [ ]:
alpha = 0.001
epsilon = 10**(-9)

In [ ]:
x_initial = np.random.uniform(0,1, n)
S_init = convert_x_to_S(x_initial)

In [ ]:
print(x_initial,S_init)

In [ ]:
_,_,x_final = gradient_ascent(F, x_initial, f_polynomial, n, alpha, t, epsilon)
S_final_grad_ascent = convert_x_to_S(x_final)

In [ ]:
print(S_final_grad_ascent)

In [ ]:
max_val,S_final_actual_max = actual_max(f_polynomial, n)

In [ ]:
max_val,S_final_actual_max

For the polynomial function the gradient ascent doesn't give the same output as the exact method